In [54]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [55]:
def load_ds(path):
    with open(path) as f:
        data = f.read()
        sub_data_list = data.split("\n\n\n")
        tags_list = []
        data = []
        for s in sub_data_list:
            detail: list = s.split("\n")
            if not detail or len(detail) < 2:
                continue
            tag = detail[0]
            tags_list.append(int(tag in ["normal", "ok"]))
            this_section_detail = []
            for row in detail[1:]:
                row_list = row.split("\t")[1:]
                if row_list:
                    this_section_detail.append(row_list)
            if this_section_detail:
                data.append(this_section_detail)
    return data, tags_list


tag_list = []
data_list = []

for i in range(1, 6):
    data, tags = load_ds(path=f'lp{i}.data')
    data_list.extend(data)
    tag_list.extend(tags)
    
for i in data_list:
    assert len(i) == 15
    for j in i:
        assert len(j) == 6

data = np.array(data_list, dtype=int)
tags = np.array(tag_list)
data = data / np.max(data, axis=0)

In [ ]:
data.shape

(463, 15, 6)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(data, tags, test_size=0.3, random_state=22)
X_test , X_val, y_test, y_val =train_test_split(X_test,y_test,test_size=0.33,random_state=22)

**elman**

In [58]:
elman_model = keras.models.Sequential()
elman_model.add(keras.layers.Input(shape=(15,6)))
elman_model.add(keras.layers.SimpleRNN(10))
elman_model.add(keras.layers.Dense(2,activation="softmax"))
elman_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
elman_model.fit(X_train,y_train,epochs=50)

Epoch 1/50
11/11 [==============================] - 1s 6ms/step - loss: 0.7055 - acc: 0.5957
Epoch 2/50
11/11 [==============================] - 0s 7ms/step - loss: 0.6520 - acc: 0.6605
Epoch 3/50
11/11 [==============================] - 0s 5ms/step - loss: 0.6221 - acc: 0.6759
Epoch 4/50
11/11 [==============================] - 0s 5ms/step - loss: 0.6018 - acc: 0.6543
Epoch 5/50
11/11 [==============================] - 0s 5ms/step - loss: 0.5831 - acc: 0.6574
Epoch 6/50
11/11 [==============================] - 0s 6ms/step - loss: 0.5680 - acc: 0.6975
Epoch 7/50
11/11 [==============================] - 0s 5ms/step - loss: 0.5558 - acc: 0.6975
Epoch 8/50
11/11 [==============================] - 0s 5ms/step - loss: 0.5465 - acc: 0.7068
Epoch 9/50
11/11 [==============================] - 0s 5ms/step - loss: 0.5401 - acc: 0.6728
Epoch 10/50
11/11 [==============================] - 0s 5ms/step - loss: 0.5357 - acc: 0.7099
Epoch 11/50
11/11 [==============================] - 0s 6ms/step - lo

In [59]:
elman_model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.3222 - acc: 0.8387


[0.3222469091415405, 0.8387096524238586]

**jordan**

In [70]:
class jordanRNN(keras.layers.Layer):

    def __init__(self, run_units,**kwargs):
        super().__init__(**kwargs)
        self.run_units = run_units
        self.state_size = run_units
        

    def build(self, input_shapes):

        self.W_xh = self.add_weight(shape=(input_shapes[-1], self.run_units),initializer='uniform',
                                      name='W_xh')
        self.W_hh = self.add_weight(shape=(self.run_units, self.run_units),initializer='uniform',name='W_hh')
        self.W_hy = self.add_weight(shape=(self.run_units,self.run_units))
        self.built=True


    def call(self, inputs, states):
        prev_output = states[0]
        h=tf.keras.activations.relu(tf.keras.backend.dot(inputs, self.W_xh) + tf.keras.backend.dot(prev_output, self.W_hh))
        output = tf.keras.activations.relu(tf.keras.backend.dot(h, self.W_hy))
        return output, [output]

    def get_config(self):
        config = super().get_config()
        config.update({
            "run_units": self.run_units,
        })
        return config
cell = jordanRNN(64)
RNN_layer = keras.layers.RNN(cell)


jordan_model = keras.models.Sequential()
jordan_model.add(keras.Input(shape=(15, 6)))
jordan_model.add(RNN_layer)
jordan_model.add(keras.layers.Dense(2, activation='softmax'))

In [71]:
jordan_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy" , metrics=["accuracy"])

In [75]:
jordan_model.fit(X_train, y_train, epochs=60)

Epoch 1/60
11/11 [==============================] - 0s 14ms/step - loss: 0.3558 - accuracy: 0.7809
Epoch 2/60
11/11 [==============================] - 0s 10ms/step - loss: 0.3541 - accuracy: 0.7809
Epoch 3/60
11/11 [==============================] - 0s 15ms/step - loss: 0.3506 - accuracy: 0.7809
Epoch 4/60
11/11 [==============================] - 0s 13ms/step - loss: 0.3393 - accuracy: 0.7932
Epoch 5/60
11/11 [==============================] - 0s 11ms/step - loss: 0.3106 - accuracy: 0.8210
Epoch 6/60
11/11 [==============================] - 0s 15ms/step - loss: 0.3092 - accuracy: 0.8210
Epoch 7/60
11/11 [==============================] - 0s 15ms/step - loss: 0.3024 - accuracy: 0.8272
Epoch 8/60
11/11 [==============================] - 0s 7ms/step - loss: 0.3019 - accuracy: 0.8272
Epoch 9/60
11/11 [==============================] - 0s 6ms/step - loss: 0.3014 - accuracy: 0.8302
Epoch 10/60
11/11 [==============================] - 0s 7ms/step - loss: 0.3010 - accuracy: 0.8302
Epoch 11/60


In [68]:
jordan_model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 4ms/step - loss: 0.2902 - accuracy: 0.8387


[0.2901843190193176, 0.8387096524238586]

In [89]:
model1 = keras.models.Sequential()
model1.add(keras.layers.Input(shape=(15,6)))
model1.add(keras.layers.SimpleRNN(20))
model1.add(keras.layers.Dense(2,activation="softmax"))

model2 = keras.models.Sequential()
model2.add(keras.layers.Input(shape=(15,6)))
model2.add(keras.layers.SimpleRNN(30))
model2.add(keras.layers.Dense(2,activation="softmax"))


model3 = keras.models.Sequential()
model3.add(keras.layers.Input(shape=(15,6)))
model3.add(keras.layers.SimpleRNN(50))
model3.add(keras.layers.Dense(2,activation="softmax"))

In [38]:
models = [model1, model2, model3] #stacking individual models in a list
model_input = tf.keras.Input(shape=(15,6)) #takes a list of tensors as input, all of the same shape
model_outputs = [model(model_input) for model in models] #collects outputs of models in a list
ensemble_output = tf.keras.layers.Average()(model_outputs) #averaging outputs
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.summary() #prints a comprehensive summary of the Keras model

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 15, 6)]      0           []                               
                                                                                                  
 sequential_11 (Sequential)     (None, 2)            582         ['input_22[0][0]']               
                                                                                                  
 sequential_12 (Sequential)     (None, 2)            1172        ['input_22[0][0]']               
                                                                                                  
 sequential_13 (Sequential)     (None, 2)            2952        ['input_22[0][0]']               
                                                                                            

In [ ]:
ensemble_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
ensemble_model.fit(X_train,y_train,epochs=50)

Epoch 1/50
11/11 [==============================] - 2s 9ms/step - loss: 0.2073 - acc: 0.9444
Epoch 2/50
11/11 [==============================] - 0s 10ms/step - loss: 0.1709 - acc: 0.9383
Epoch 3/50
11/11 [==============================] - 0s 9ms/step - loss: 0.1794 - acc: 0.9568
Epoch 4/50
11/11 [==============================] - 0s 10ms/step - loss: 0.1637 - acc: 0.9691
Epoch 5/50
11/11 [==============================] - 0s 9ms/step - loss: 0.1693 - acc: 0.9290
Epoch 6/50
11/11 [==============================] - 0s 10ms/step - loss: 0.1515 - acc: 0.9444
Epoch 7/50
11/11 [==============================] - 0s 9ms/step - loss: 0.1375 - acc: 0.9630
Epoch 8/50
11/11 [==============================] - 0s 9ms/step - loss: 0.1230 - acc: 0.9352
Epoch 9/50
11/11 [==============================] - 0s 8ms/step - loss: 0.1200 - acc: 0.9383
Epoch 10/50
11/11 [==============================] - 0s 9ms/step - loss: 0.1163 - acc: 0.9599
Epoch 11/50
11/11 [==============================] - 0s 12ms/step 

In [ ]:
ensemble_model.evaluate(X_test,y_test)

3/3 [==============================] - 1s 9ms/step - loss: 0.1104 - acc: 0.9785


[0.11036979407072067, 0.9784946441650391]

In [78]:
cell = jordanRNN(128)
RNN_layer = keras.layers.RNN(cell)
jordan_model1 = keras.models.Sequential()
jordan_model1.add(keras.Input(shape=(15, 6)))
jordan_model1.add(RNN_layer)
jordan_model1.add(keras.layers.Dense(2, activation='softmax'))

cell = jordanRNN(64)
RNN_layer = keras.layers.RNN(cell)
RNN_layer = keras.layers.RNN(cell)
jordan_model2 = keras.models.Sequential()
jordan_model2.add(keras.Input(shape=(15, 6)))
jordan_model2.add(RNN_layer)
jordan_model2.add(keras.layers.Dense(2, activation='softmax'))

cell = jordanRNN(32)
RNN_layer = keras.layers.RNN(cell)
RNN_layer = keras.layers.RNN(cell)
jordan_model3 = keras.models.Sequential()
jordan_model3.add(keras.Input(shape=(15, 6)))
jordan_model3.add(RNN_layer)
jordan_model3.add(keras.layers.Dense(2, activation='softmax'))


In [94]:
models = [jordan_model1, jordan_model2, jordan_model3] #stacking individual models in a list
model_input = tf.keras.Input(shape=(15,6)) #takes a list of tensors as input, all of the same shape
model_outputs = [model(model_input) for model in models] #collects outputs of models in a list
ensemble_output = tf.keras.layers.Average()(model_outputs) #averaging outputs
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.summary() #prints a comprehensive summary of the Keras model


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, 15, 6)]      0           []                               
                                                                                                  
 sequential_33 (Sequential)     (None, 2)            33794       ['input_54[0][0]']               
                                                                                                  
 sequential_34 (Sequential)     (None, 2)            8706        ['input_54[0][0]']               
                                                                                                  
 sequential_35 (Sequential)     (None, 2)            2306        ['input_54[0][0]']               
                                                                                           

In [95]:
ensemble_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
ensemble_model.fit(X_train,y_train,epochs=50)

Epoch 1/50
11/11 [==============================] - 2s 14ms/step - loss: 0.3646 - acc: 0.7253
Epoch 2/50
11/11 [==============================] - 0s 14ms/step - loss: 0.4503 - acc: 0.7253
Epoch 3/50
11/11 [==============================] - 0s 16ms/step - loss: 0.3795 - acc: 0.7253
Epoch 4/50
11/11 [==============================] - 0s 14ms/step - loss: 0.3833 - acc: 0.7253
Epoch 5/50
11/11 [==============================] - 0s 14ms/step - loss: 0.3882 - acc: 0.7253
Epoch 6/50
11/11 [==============================] - 0s 15ms/step - loss: 0.3882 - acc: 0.7253
Epoch 7/50
11/11 [==============================] - 0s 14ms/step - loss: 0.3881 - acc: 0.7253
Epoch 8/50
11/11 [==============================] - 0s 14ms/step - loss: 0.3879 - acc: 0.7253
Epoch 9/50
11/11 [==============================] - 0s 16ms/step - loss: 0.3878 - acc: 0.7253
Epoch 10/50
11/11 [==============================] - 0s 14ms/step - loss: 0.3877 - acc: 0.7253
Epoch 11/50
11/11 [==============================] - 0s 14m

In [97]:
ensemble_model.evaluate(X_test,y_test)

3/3 [==============================] - 1s 7ms/step - loss: 0.4170 - acc: 0.7204


[0.41703662276268005, 0.7204301357269287]

In [101]:
models = [ jordan_model2, model2 ,model3] #stacking individual models in a list
model_input = tf.keras.Input(shape=(15,6)) #takes a list of tensors as input, all of the same shape
model_outputs = [model(model_input) for model in models] #collects outputs of models in a list
ensemble_output = tf.keras.layers.Average()(model_outputs) #averaging outputs
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.summary() #prints a comprehensive summary of the Keras model

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_57 (InputLayer)          [(None, 15, 6)]      0           []                               
                                                                                                  
 sequential_34 (Sequential)     (None, 2)            8706        ['input_57[0][0]']               
                                                                                                  
 sequential_37 (Sequential)     (None, 2)            1172        ['input_57[0][0]']               
                                                                                                  
 sequential_38 (Sequential)     (None, 2)            2952        ['input_57[0][0]']               
                                                                                           

In [103]:
ensemble_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
ensemble_model.fit(X_train,y_train,epochs=50)

Epoch 1/50
11/11 [==============================] - 2s 10ms/step - loss: 0.2587 - acc: 0.9167
Epoch 2/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2680 - acc: 0.9167
Epoch 3/50
11/11 [==============================] - 0s 11ms/step - loss: 0.2494 - acc: 0.9321
Epoch 4/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2474 - acc: 0.9352
Epoch 5/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2457 - acc: 0.9352
Epoch 6/50
11/11 [==============================] - 0s 11ms/step - loss: 0.2450 - acc: 0.9352
Epoch 7/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2439 - acc: 0.9352
Epoch 8/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2386 - acc: 0.9383
Epoch 9/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2440 - acc: 0.9383
Epoch 10/50
11/11 [==============================] - 0s 10ms/step - loss: 0.2445 - acc: 0.9352
Epoch 11/50
11/11 [==============================] - 0s 10m

In [105]:
ensemble_model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.2873 - acc: 0.9247


[0.28726017475128174, 0.9247311949729919]